<a href="https://www.kaggle.com/code/aashutoshh01/face-detection?scriptVersionId=191269804" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Face Detection Model**

* **Aashutosh Joshi**
* **Indian Institute of Technology, Kharagpur**

# **Import Module**

In [ ]:
import os
import numpy as np
import pandas as pd
import shutil
import cv2
import random
import matplotlib.pyplot as plt
import copy
import wandb

# **Working With Directory**

By structuring the file paths in the following way, we code can dynamically adapt to different working directories and ensures that paths to data directories are consistently and correctly formed.
* **curr_path** holds the current working directory path.
* **imgtrainpath**, **imgvalpath**, and **imgtestpath** are paths to the directories where training, validation, and testing images are stored, respectively.
* **labeltrainpath**, **labelvalpath**, and **labeltestpath** are paths to the directories where training, validation, and testing labels are stored, respectively.


In [ ]:
curr_path=os.getcwd()
imgtrainpath = os.path.join(curr_path,'images','train')
imgvalpath=os.path.join(curr_path,'images','validation')
imgtestpath=os.path.join(curr_path,'images','test')

labeltrainpath=os.path.join(curr_path,'labels','train')
labelvalpath=os.path.join(curr_path,'labels','validation')
labeltestpath=os.path.join(curr_path,'labels','test')

In [ ]:
bs=' ' # blank-space
class_id=0 # id for face
newline='\n' # new line character
extension='.txt' # extension for text file

By defining the following paths below,we can easily refer to our data and labels throughout our code without hardcoding the directory structure multiple times.

In [ ]:
data_path='/kaggle/input/human-faces-object-detection'
labels_path = os.path.join(curr_path, 'face_labels')

By running the following line of code, we ensure that the directory for storing face label files exists, which is useful for organizing our data and labels in a consistent directory structure.

In [ ]:
os.makedirs(labels_path)

The following function is useful for inspecting the contents of a directory, allowing us to programmatically access and manipulate the files and subdirectories within it.

In [ ]:
os.listdir(data_path)

By defining the following paths in the cell below, we can easily access and manipulate the images and annotations in our dataset without hardcoding the directory structure multiple times, ensuring our code remains flexible and maintainable.
* **img_path** is constructed to hold the path to the '**images**' subdirectory within our dataset directory, which is where our image files are stored.
* **raw_annotations_path** is constructed to hold the path to the '**faces.csv**' file within our dataset directory, which is where our raw annotations or metadata are stored.

In [ ]:
img_path=os.path.join(data_path, 'images')
raw_annotations_path=os.path.join(data_path, 'faces.csv')

The following cell is useful for
* Iterating over the images in the directory.
* Processing each image file, such as loading the images into memory, performing operations on them, or using them for training a model.

In [ ]:
face_list=os.listdir(img_path)

In [ ]:
face_list[:5]

In [ ]:
data_len=len(face_list)
data_len

In [ ]:
random.shuffle(face_list)
face_list[:5]

# **Train Test Split**

In [ ]:
train_split=0.8
val_split=0.1
test_split=0.1

imgtrain_list=face_list[:int(data_len*train_split)]
imgval_list=face_list[int(data_len*train_split):int(data_len*(train_split+val_split))]
imgtest_list=face_list[int(data_len*(train_split+val_split)):]

In [ ]:
imgtest_list[:5]

In [ ]:
len(imgtrain_list), len(imgval_list), len(imgtest_list)

This function below is useful for changing the file extension of filenames while keeping the base name intact, such as when converting files from one format to another.
* **change_extension(file)**: Takes a file path as input, removes its current extension, and replaces it with a new extension defined by the **extension** variable.
* For instance, if you call **change_extension('image.jpg')** and extension is '**.png**', the function will return '**image.png**'.

In [ ]:
def change_extension(file):
    basename=os.path.splitext(file)[0]
    filename=basename+extension
    return filename

The approach in the cell below is useful for creating corresponding label files or other related files that follow a different naming convention or file format.
* **labeltrain_list**: A list of filenames for the training set with the new file extension.
* **labelval_list**: A list of filenames for the validation set with the new file extension.
* **labeltest_list**: A list of filenames for the testing set with the new file extension.

In [ ]:
labeltrain_list = list(map(change_extension, imgtrain_list)) 
labelval_list = list(map(change_extension, imgval_list)) 
labeltest_list = list(map(change_extension, imgtest_list)) 

In [ ]:
len(labeltrain_list), len(labelval_list), len(labeltest_list)
labeltest_list[:5] 

# **Reading a CSV file into the DataFrame**

In the cell below
* **pd.read_csv(raw_annotations_path)**: Reads the CSV file specified by **raw_annotations_path** and creates a DataFrame from it.
* **raw_annotations**: Holds the DataFrame with the contents of the CSV file, allowing for further manipulation and analysis.

In [ ]:
raw_annotations=pd.read_csv(raw_annotations_path)
raw_annotations

The new columns in the following cell are useful for tasks like training object detection models, where bounding box center coordinates and dimensions are often required.
* **raw_annotations['x_centre']**: Contains the center x-coordinate of the bounding boxes.
* **raw_annotations['y_centre']**: Contains the center y-coordinate of the bounding boxes.
* **raw_annotations['bb_width']**: Contains the width of the bounding boxes.
* **raw_annotations['bb_height']**: Contains the height of the bounding boxes.

In [ ]:
raw_annotations['x_centre']=0.5*(raw_annotations['x0']+raw_annotations['x1'])
raw_annotations['y_centre']=0.5*(raw_annotations['y0']+raw_annotations['y1'])
raw_annotations['bb_width']=raw_annotations['x1']-raw_annotations['x0']
raw_annotations['bb_height']=raw_annotations['y1']-raw_annotations['y0']
raw_annotations

Normalizing the bounding box coordinates and dimensions to be between 0 and 1 is a common preprocessing step in many computer vision tasks, particularly for object detection models. It helps standardize the data and ensures that the bounding box values are relative to the size of the image, which can improve the model's performance and consistency.

In the cell below:
* **raw_annotations['xcentre_scaled']**: Contains the normalized center x-coordinate of the bounding boxes, scaled relative to the width of the image.
* **raw_annotations['ycentre_scaled']**: Contains the normalized center y-coordinate of the bounding boxes, scaled relative to the height of the image.
* **raw_annotations['width_scaled']**: Contains the normalized width of the bounding boxes, scaled relative to the width of the image.
* **raw_annotations['height_scaled']**: Contains the normalized height of the bounding boxes, scaled relative to the height of the image.

In [ ]:
raw_annotations['xcentre_scaled']=raw_annotations['x_centre']/raw_annotations['width']
raw_annotations['ycentre_scaled']=raw_annotations['y_centre']/raw_annotations['height']
raw_annotations['width_scaled']=raw_annotations['bb_width']/raw_annotations['width']
raw_annotations['height_scaled']=raw_annotations['bb_height']/raw_annotations['height']
raw_annotations

In [ ]:
len(raw_annotations['image_name'].unique())

# **Preprocessing**

In the cell below:
* **imgs**: Holds a **GroupBy** object created by grouping **raw_annotations** by the **image_name** column.
* This allows you to perform operations on subsets of data that share the same **image_name**, making it easier to analyze and process data associated with each image.

In [ ]:
imgs=raw_annotations.groupby('image_name') 

In the cell below:
* **Purpose**: The code processes bounding box annotations for each image and writes them into a text file where each line represents an object annotation in the format required.
* **File Creation**: For each image, it generates a label file with annotations, with each annotation line containing normalized coordinates and dimensions.

In [ ]:
#Loop Through Groups:
for image in imgs:
    
    #Get DataFrame for Each Image:
    img_df=imgs.get_group(image[0])
    
    #Generate Filename for the Label File:
    basename=os.path.splitext(image[0])[0]
    txt_file=basename+extension
    filepath=os.path.join(labels_path, txt_file)
    
    #Prepare Lines for the Label File:
    lines=[]
    i=1
    
    #Generate Annotation Lines:
    for index,row in img_df.iterrows():
        
        #Check if the current row is not the last one
        if i!=len(img_df):
            line=str(class_id)+bs+str(row['xcentre_scaled'])+bs+str(row['ycentre_scaled'])+bs+str(row['width_scaled'])+bs+str(row['height_scaled'])+newline
            lines.append(line)
        else:
            line=str(class_id)+bs+str(row['xcentre_scaled'])+bs+str(row['ycentre_scaled'])+bs+str(row['width_scaled'])+bs+ str(row['height_scaled'])
            lines.append(line)
        i=i+1
    
    #Write to the Label File:
    with open(filepath, 'w') as file:
        file.writelines(lines)
        

Following snippet is useful for verifying the contents of a randomly chosen label file to ensure that the annotation data is formatted correctly and has been written as expected.
* **os.listdir(labels_path)[:5]**: Lists the first 5 files or directories in the labels_path directory.
* **random_file**: Constructs the path to the 5th file in the directory
* **Reading File**: Opens and reads the content of the selected label file into the content variable.
* **content**: Contains the data from the label file, which is printed or displayed.

In [ ]:
#List Files in Directory (os.listdir):
os.listdir(labels_path)[:5]

#Select a Random File (random_file):
random_file=os.path.join(labels_path, os.listdir(labels_path)[4])

#Read the Content of the Selected File:
with open (random_file, 'r') as f:
    content=f.read()
    
#Display Content:
content

In [ ]:
def_size=640
len(os.listdir(labels_path)) 

In the following cell:
* **Purpose**: The **move_files** function moves files from a specified source directory to a destination directory.
* **File Path Construction**: Constructs full file paths for both source and destination.
* **Directory Creation**: Creates the destination directory if it does not already exist.
* **File Movement**: Moves each file and counts the number of files moved.
* **Completion Message**: Prints the total number of files transferred.

In [ ]:
def move_files(data_list, source_path, destination_path):
    
    #Initialize Counter:
    i=0
    
    #Loop Through Files:
    for file in data_list:
        
        #Construct File Paths:
        filepath=os.path.join(source_path, file)
        dest_path=os.path.join(data_path, destination_path)
        
        #Create Destination Directory if Needed:
        if not os.path.isdir(dest_path):
            os.makedirs(dest_path)
            
        #Move File:
        shutil.move(filepath, dest_path)
        
        #Update Counter:
        i=i+1
    print("Number of files transferred:", i)

Following move_images function is designed to move and resize images from a source directory to a destination directory.
* **Purpose**: The **move_images** function resizes images from a source directory and moves them to a destination directory.
* **Image Processing**: Uses OpenCV to read, resize, and save images.
* **Directory Handling**: Ensures the destination directory exists and creates it if necessary.
* **Tracking**: Counts the number of processed images and prints this count at the end.

In [ ]:
def move_images(data_list, source_path, destination_path):
    
    #Initialize Counter:
    i=0
    
    #Loop Through Files:
    for file in data_list:
        
        #Construct File Paths:
        filepath=os.path.join(source_path, file)
        dest_path=os.path.join(data_path, destination_path)
        
        #Create Destination Directory if Needed:
        if not os.path.isdir(dest_path):
            os.makedirs(dest_path)
            
        #Resize and Save Image:
        finalimage_path=os.path.join(dest_path, file)
        img_resized=cv2.resize(cv2.imread(filepath), (def_size, def_size))
        cv2.imwrite(finalimage_path, img_resized)
        
        #Update Counter:
        i=i+1
    print("Number of files transferred:", i)

The step in the cell below is crucial for preparing the dataset for machine learning or computer vision tasks, ensuring that all training images are uniformly resized and organized.
* **Purpose**: This line of code initiates the process of resizing and moving training images from their original location to a designated training directory.
* **Data Handling**: Ensures that images are properly resized and stored in the correct location for training purposes.
* **Function Integration**: Utilizes the **move_images** function to automate and streamline the process of preparing training data.

In [ ]:
move_images(imgtrain_list, img_path, imgtrainpath)

In [ ]:
move_images(imgval_list, img_path, imgvalpath)
move_images(imgtest_list, img_path, imgtestpath)

In [ ]:
move_files(labeltrain_list, labels_path, labeltrainpath)
move_files(labelval_list, labels_path, labelvalpath)
move_files(labeltest_list, labels_path, labeltestpath)

In the following cell:
* **Counting Files**: The **len(os.listdir(labels_path))** part of the code calculates how many items are in the **labels_path** directory.
* **Deleting Directory**: The **shutil.rmtree(labels_path)** part of the code removes the **labels_path** directory and all its contents, effectively cleaning up or resetting the directory state.

In [ ]:
len(os.listdir(labels_path)) 
shutil.rmtree(labels_path)

The configuration file in the cell below is typically used by machine learning frameworks to load datasets and understand the structure and classes of the data.

The config_lines list contains all the necessary lines to create a configuration file with:
* Paths to training, validation, and test datasets.
* Class definitions for object detection or classification tasks.

In [ ]:
ln_1='# Train/val/test sets'+newline
ln_2='train: ' +"'"+imgtrainpath+"'"+newline
ln_3='val: ' +"'" + imgvalpath+"'"+newline
ln_4='test: ' +"'" + imgtestpath+"'"+newline
ln_5=newline
ln_6='# Classes'+newline
ln_7='names:'+newline
ln_8='  0: face'
config_lines=[ln_1, ln_2, ln_3, ln_4, ln_5, ln_6, ln_7, ln_8]

In the following cell:
* **config_path**: This variable holds the full path to the **config.yaml** file in the current working directory.
* **Purpose**: Used to specify where the configuration file is located for writing or reading configuration settings.

In [ ]:
config_path=os.path.join(curr_path, 'config.yaml')
config_path

In the cell below:
* **Purpose**: The code writes the configuration settings to a file named **config.yaml**.
* **File Handling**: Opens the file in write mode and ensures it is closed properly after writing.
* **Writing Lines**: Uses the writelines method to write a list of lines to the file.

In [ ]:
with open(config_path, 'w') as f:
    f.writelines(config_lines)

# **Image Visualization**

The **get_bbox_from_label** function below:

* Reads a text file containing bounding box data.
* Parses the data to extract bounding box coordinates.
* Converts the coordinates from normalized values to pixel values.
* Constructs a list of bounding box vertices.
* Returns the list as a tuple.

In [ ]:
#Function Definition and Initialization
def get_bbox_from_label(text_file_path):
    
    #Opening and Reading the File
    bbox_list=[]
    with open(text_file_path, "r") as file:
        for line in file:
            
            #Processing Each Line
            _,x_centre,y_centre,width,height=line.strip().split(" ")
            x1=(float(x_centre)+(float(width)/2))*def_size
            x0=(float(x_centre)-(float(width)/2))*def_size
            y1=(float(y_centre)+(float(height)/2))*def_size
            y0=(float(y_centre)-(float(height)/2))*def_size
            
            #Creating Vertices and Appending to List
            vertices=np.array([[int(x0), int(y0)], [int(x1), int(y0)], 
                               [int(x1),int(y1)], [int(x0),int(y1)]])
            bbox_list.append(vertices)      
     
    #Return the Bounding Boxes
    return tuple(bbox_list)

In [ ]:
red=(255,0,0) 

In the cell below:

**Figure Setup**: Initializes a large figure with dimensions 30x30 inches.

**Loop**: Iterates through a specified range to create pairs of subplots.
* Randomly selects an image and its corresponding label.
* Reads the image and creates a deep copy.
* Extracts bounding box coordinates from the label.
* Displays the original image in one subplot.
* Draws bounding boxes on the copied image and displays it in the adjacent subplot.

In [ ]:
#Set Up the Figure
plt.figure(figsize=(30,30))

#Loop to Process and Display Images
for i in range(1,8,2):
    
    #Random Selection:
    k=random.randint(0, len(imgtrain_list)-1)
    
    #Paths for Image and Label:
    img_path=os.path.join(imgtrainpath, imgtrain_list[k])
    label_path=os.path.join(labeltrainpath, labeltrain_list[k])
    
    #Bounding Box Extraction:
    bbox=get_bbox_from_label(label_path)
    
    #Image Reading and Copy:
    image=cv2.imread(img_path)
    image_copy=copy.deepcopy(image)
    
    #Display Original Image
    ax=plt.subplot(4, 2, i)
    plt.imshow(image) 
    plt.xticks([])
    plt.yticks([])
    
    #Draw Bounding Box and Display Annotated Image
    cv2.drawContours(image_copy, bbox, -1, red, 2) 
    ax=plt.subplot(4, 2, i+1)
    plt.imshow(image_copy) 
    plt.xticks([])
    plt.yticks([])

# **Importing Model**

**ultralytics** is the name of the package we want to install. Ultralytics is known for developing YOLO (You Only Look Once) models for object detection.

In [ ]:
!pip install ultralytics

Cell below imports the **YOLO** class from the **ultralytics** package

In [ ]:
from ultralytics import YOLO

The code snippet below initializes a **YOLOv8** model using a configuration file ('**yolov8n.yaml**') and loads pre-trained weights ('**yolov8n.pt**'). This setup allows us to perform object detection tasks with the pre-trained YOLOv8 model.

In [ ]:
model=YOLO('yolov8n.yaml').load('yolov8n.pt')

# **Training The YOLO Model**

Following cell initiates the training process of the YOLO model with specific parameters. Here's a detailed explanation of each part:

**Breakdown of Code**

**model.train()**
* **train()**: This is a method of the YOLO class that starts the training process for the model.
**Parameters:**
* **data=config_path**: **data** specifies the path to the data configuration file, which contains information about the dataset, including paths to training, validation, and test sets, as well as class names. Here, **config_path** is the variable holding the path to your configuration file ('**config.yaml**').
* **epochs=100**: **epochs** specifies the number of training epochs. An epoch is one complete pass through the entire training dataset. Here, the model will train for 100 epochs.
* **resume=True**: **resume** indicates whether to resume training from the last checkpoint. If set to **True**, training will continue from where it left off, using saved checkpoints.
* **iou=0.5**: **iou** is Intersection over Union threshold for evaluation. It is used to determine whether a predicted bounding box is considered a true positive. A higher IoU threshold means stricter evaluation criteria.
* **conf=0.001**: **conf** is confidence threshold for object detection. It sets the minimum confidence level required for a prediction to be considered valid. Lower values allow more detections but can increase false positives.

In [ ]:
results=model.train(data=config_path, epochs=100, resume=True, iou=0.5, conf=0.001)

# **Evaluation And Plots**

Code snippet below is used to visualize the training results of the **YOLO** model by displaying an image file named results.png, which typically contains training metrics such as loss and accuracy curves. 

In [ ]:
#Create a large figure for plotting
plt.figure(figsize=(30,30))

#Construct the path to the training results
trainingresult_path=os.path.join(curr_path, 'runs', 'detect', 'train')

#Read the results image
results_png=cv2.imread(os.path.join(trainingresult_path,'results.png'))

#Display the results image
plt.imshow(results_png)

The function **evaluate_map50** is designed to evaluate a trained **YOLO** model on a specified dataset and calculate the **mean Average Precision (mAP)** at an **IoU** threshold of **0.5**.

In [ ]:
#Function Definition
def evaluate_map50(trainedmodel, data_path, dataset='val'):
    
    #Evaluate the Model
    metrics=trainedmodel.val(data=data_path, split=dataset)
    
    #Calculate mAP@50
    map50=round(metrics.box.map50, 3)
    
    #Print the Results
    print("The mAP of model on {0} dataset is {1}".format(dataset,map50))
    
    #Return the Results
    return metrics, map50

The **display_curves** function is designed to display various plots generated during the training and evaluation of a **YOLO** model. The plots include the **Precision curve**, **Recall curve**, **Precision-Recall curve**, **F1 curve**, and the **Confusion matrix**. These plots provide insights into the model's performance and behavior.

In [ ]:
#Function Definition
def display_curves(root_path):
    
    #Create a Figure
    plt.figure(figsize=(50,50))
    
    #Display the Precision Curve
    p_curve=cv2.imread(os.path.join(root_path,"P_curve.png"))
    ax=plt.subplot(5,1,1)
    print(p_curve)
    plt.imshow(p_curve)
    
    #Display the Recall Curve
    r_curve=cv2.imread(os.path.join(root_path,"R_curve.png"))
    ax=plt.subplot(5,1,2)
    plt.imshow(r_curve)
    
    #Display the Precision-Recall Curve
    pr_curve=cv2.imread(os.path.join(root_path,"PR_curve.png"))
    ax=plt.subplot(5,1,3)
    plt.imshow(pr_curve)
    
    #Display the F1 Curve
    f1_curve=cv2.imread(os.path.join(root_path,"F1_curve.png"))
    ax=plt.subplot(5,1,4)
    plt.imshow(f1_curve)
    
    #Display the Confusion Matrix
    confusion_matrix=cv2.imread(os.path.join(root_path,"confusion_matrix.png"))
    ax=plt.subplot(5,1,5)
    plt.imshow(confusion_matrix)

Following line of code allows us to assess the performance of our **YOLO** model on the training dataset, providing insight into how well the model has learned to detect objects during the training phase.

In [ ]:
train_metrics, train_map50=evaluate_map50(model, config_path, dataset='train')

In the cell below the variable **train_path** is set to a directory that contains the evaluation results for a specific training run (**'train2'**). This directory is used to store results from evaluating the model on the training dataset, not on a validation dataset. The path is constructed using **os.path.join** to ensure it correctly points to the location within the file system based on the current working directory.

In [ ]:
train_path=os.path.join(curr_path, 'runs', 'detect', 'train2') #val is a misnomer, it is actually measuring validation on training dataset

In the cell below visualization helps in assessing the performance of the model on the training dataset by providing a clear view of various metrics, allowing us to analyze and interpret the results of our model training and evaluation.

In [ ]:
display_curves(train_path)

Line of code in the cell below helps us to assess how well the **YOLO** model performs on the validation dataset, providing important insights into its ability to detect objects accurately when exposed to unseen data.

In [ ]:
val_metrics, val_map50=evaluate_map50(model, config_path, dataset='val')

Cell below is used to define the path to a specific directory where evaluation results or training outputs are stored.

In [ ]:
val_path=os.path.join(curr_path, 'runs', 'detect', 'train4') 

Cell below is used to evaluate the performance of a trained **YOLO** model on the test dataset.

In [ ]:
test_metrics, test_map50=evaluate_map50(model, config_path, dataset='test')

Cell below is used to define the path to a directory where evaluation results for a specific training run are stored.

In [ ]:
test_path=os.path.join(curr_path, 'runs', 'detect', 'train3') 

Following cell is used to visualize various performance metrics of a model by displaying plots that were generated during the evaluation process

In [ ]:
display_curves(test_path) 

The block of code provided is used to visualize a random set of images from the test dataset along with the predictions made by the trained **YOLO** model.

This code snippet performs the following tasks:
* **Initialize Plot**: Sets up a large figure for displaying images.
* **Select Random Image**: Chooses a random image index from the test dataset.
* **Display Actual Image**: Shows the actual image from the test dataset.
* **Make Predictions**: Uses the YOLO model to predict objects in the image.
* **Display Predictions**: Shows the image with bounding boxes and labels for the detected objects.

In [ ]:
#Plot Initialization
plt.figure(figsize=(60,60))

#Random Image Selection
m=random.randint(0, 150) 

#Plotting Loop
for i in range(1,8,2):
    test_image=os.path.join(imgtestpath, os.listdir(imgtestpath)[m])
    ax=plt.subplot(4,2,i)
    
    #Display actual image
    plt.imshow(cv2.imread(test_image)) 
    plt.xticks([])
    plt.yticks([])
    plt.title("Actual image", fontsize = 40)
    
    #Predict 
    res = model(test_image)
    res_plotted = res[0].plot()
    ax=plt.subplot(4,2,i+1)
    
    #Display image with predictions
    plt.imshow(res_plotted)
    plt.title("Image with predictions", fontsize = 40)
    plt.xticks([])
    plt.yticks([])
    m=m+1